<a href="https://colab.research.google.com/github/senjoyee/udemy_langchain_python/blob/main/langchain_python_bootcamp_4_34_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install -qU langchain-openai

In [59]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

Set API Key

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Initialize LLM

In [11]:
llm = ChatOpenAI()

Define templates

In [12]:
beginner_template = """You are a physics teacher who is really focused on
beginners and explaining complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your question:\n{input}"""

expert_template = """You are a physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a PhD in Physics.
Here is your question:\n{input}"""

Define prompt information

In [13]:
prompt_infos = [
    {"name": "beginner physics", "description": "Answers basic physics questions", "template": beginner_template},
    {"name": "advanced physics", "description": "Answers advanced physics questions", "template": expert_template}
]

Create destination chains using LCEL

In [90]:
destination_chains = {p_info["name"]: LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template(p_info["template"])) for p_info in prompt_infos }

Default Chain

In [91]:
default_prompt = ChatPromptTemplate.from_template("{input}")

In [92]:
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [93]:
default_chain

LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7a1a214d7d00>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a1a214d7b50>, openai_api_key=SecretStr('**********'), openai_proxy=''))

Router Template

In [94]:
destinations_str = "\n".join([f'{p["name"]}: {p["description"]}' for p in prompt_infos])

In [95]:
print(destinations_str)

beginner physics: Answers basic physics questions
advanced physics: Answers advanced physics questions


In [96]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

In [ ]:
print(router_template)

In [87]:
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser())

Create router chain

In [98]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

Create multi-prompt chain

In [99]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [100]:
chain.invoke("What is a magnet")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'What is a magnet'}
> Finished chain.


{'input': 'What is a magnet',
 'text': 'A magnet is a material that has the ability to attract certain types of objects, such as iron or steel. Magnets have two main properties: they have a north pole and a south pole. Like poles repel each other, while opposite poles attract each other. This is why magnets can stick to metal objects or attract other magnets. Magnets are used in many everyday items, such as refrigerator magnets, compasses, and even in technology like speakers and hard drives.'}

Key consideration is that when using a MultiPromptChain, the router chain, destination chain & the default chain, each need to be an instance of LLMChain